In [53]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

def doviz_cek():
    url = "https://canlidoviz.com/"
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    isimler = soup.find_all("span", class_="whitespace-nowrap items-center text-base uppercase text-blue50 theme-light:text-wblue50 !text-[15px]")
    degerler = soup.find_all("span", class_="max-w-min text-2xl font-semibold text-white theme-light:text-darkBg px-0.5")
    artislar = soup.find_all("span", class_="text-[10px] text-gray150 data-[incr=true]:text-green50 data-[decr=true]:text-red50")

    doviz_listesi = []

    for isim, deger, artis in zip(isimler, degerler, artislar):
        doviz_listesi.append({
            "isim": isim.text.strip(),
            "deger": deger.text.strip(),
            "degisim": artis.text.strip()
        })

    return doviz_listesi


def borsa_veri_cek():
    url = "https://canlidoviz.com/"
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    isimler = soup.find_all("span", class_="truncate text-gray800 theme-light:text-darkBg text-[15px] mr-2 leading-[20px]")
    degerler = soup.find_all("span", class_="text-[15px] px-1 theme-light:text-darkBg")
    artislar = soup.find_all("span", class_="text-right text-xs text-gray150 data-[incr=true]:text-green50 data-[incr=true]:theme-light:text-incrbg data-[decr=true]:text-red50")

    borsa_listesi = []

    for isim, deger, artis in zip(isimler, degerler, artislar):
        borsa_listesi.append({
            "isim": isim.text.strip(),
            "deger": deger.text.strip(),
            "degisim": artis.text.strip()
        })

    return borsa_listesi


def degisim_puani(degisim_str):
    try:
        degisim_num = float(degisim_str.replace('%', '').replace(',', '.'))
        return degisim_num
    except:
        return 0


def piyasa_yorumu(puan):
    if puan > 0.1:
        return "Piyasa genel olarak olumlu hareket ediyor."
    elif puan < -0.1:
        return "Piyasa genel olarak olumsuz hareket ediyor."
    else:
        return "Piyasa yatay seyrediyor."


def ana_fonksiyon():
    dovizler = doviz_cek()
    borsalar = borsa_veri_cek()

    doviz_puanlar = [degisim_puani(d['degisim']) for d in dovizler]
    borsa_puanlar = [degisim_puani(b['degisim']) for b in borsalar]

    ortalama_doviz_puani = sum(doviz_puanlar) / len(doviz_puanlar) if doviz_puanlar else 0
    ortalama_borsa_puani = sum(borsa_puanlar) / len(borsa_puanlar) if borsa_puanlar else 0

    doviz_yorum = piyasa_yorumu(ortalama_doviz_puani)
    borsa_yorum = piyasa_yorumu(ortalama_borsa_puani)

    print(f"Döviz piyasası ortalama değişim puanı: {ortalama_doviz_puani:.4f}")
    print(f"Borsa piyasası ortalama değişim puanı: {ortalama_borsa_puani:.4f}")
    print("Döviz piyasası yorumu:", doviz_yorum)
    print("Borsa piyasası yorumu:", borsa_yorum)

    bugun = datetime.now().strftime("%Y-%m-%d")

    df_sonuc = pd.DataFrame({
        "Tarih": [bugun, bugun],
        "Piyasa": ["Döviz", "Borsa"],
        "Ortalama_Değişim_Puani": [ortalama_doviz_puani, ortalama_borsa_puani],
        "Yorum": [doviz_yorum, borsa_yorum]
    })

    df_sonuc.to_csv("piyasa_analizi.csv", index=False, encoding="utf-8-sig")
    print("✅ Ekonomi piyasa analizi başarıyla kaydedildi: data/ekonomi/piyasa_analizi.csv")


if __name__ == "__main__":
    ana_fonksiyon()


Döviz piyasası ortalama değişim puanı: 0.1467
Borsa piyasası ortalama değişim puanı: -0.1047
Döviz piyasası yorumu: Piyasa genel olarak olumlu hareket ediyor.
Borsa piyasası yorumu: Piyasa genel olarak olumsuz hareket ediyor.
✅ Ekonomi piyasa analizi başarıyla kaydedildi: data/ekonomi/piyasa_analizi.csv
